# Nearest Neighbor Benchmark Assignment

Performing an exhaustive exact k-nearest neighbor (kNN) search, commonly referred to as a brute-force search, is computationally expensive and does not scale efficiently with larger datasets. In vector search, this method involves calculating the distance between each query vector and every vector in the database. For widely used metrics like Euclidean and cosine distances, this process essentially transforms into a large-scale matrix multiplication problem.

In the time of AI, vector search or similarity search is becoming one of the hottest topics because of its application in large language models (LLM) and generative AI. In this context similarity search can be applied in many domains from detecting fraudulent transactions, recommending products to users, retrieving faces or connecting space points.
In this assignment we are exploring different method for performing knn search with the possibility of running these methods on the GPU to speed it up.

First, we need to create a conda environment with all the necessary packages.

In the knn notebook we show 4 functions for computing the k nearest neighbors: sklearn, cuml, raft and frnn. SkLearn runs on the CPU while the other three run on the GPU. First we are randomly generating a dataset and a set of queries on. The %%timeit decorator is used to run each cell 1000 times and record the time it takes to run each loop.

1. Generate some random data.

In [ ]:
import os
import cupy as cp
from pylibraft.common import DeviceResources
from sklearn.neighbors import NearestNeighbors as skNearestNeighbors
from cuml.neighbors import NearestNeighbors as cuNearestNeighbors
from pylibraft.neighbors.brute_force import knn
import frnn
import h5py
import urllib


n_samples = 10000
n_features = 25
n_queries = 1000
rad = 0.4
dataset = cp.random.random_sample((n_samples, n_features),
                                  dtype=cp.float32)
# Search using the built index
queries = cp.random.random_sample((n_queries, n_features),
                                  dtype=cp.float32)
k = 10

2. Run the following 4 cells and record the timing it takes to run each loop of each method.

In [ ]:
# k Nearest Neighbor from Sklearn (CPU)
%%timeit
neighbors = skNearestNeighbors(n_neighbors=k)
neighbors.fit(cp.asnumpy(dataset))
for el in cp.asnumpy(queries):
    neighbors.kneighbors([el])

In [ ]:
# k Nearest Neighbor from Rapids AI cuML (GPU)
%%timeit
cu_neighbors = cuNearestNeighbors(n_neighbors=k)
cu_neighbors.fit(dataset)
cu_neighbors.kneighbors(queries)

In [ ]:
# k Nearest Neighbor brute force from Rapids AI raft (GPU)
%%timeit
distances, neighbors = knn(dataset, queries, k)
distances = cp.asarray(distances)
neighbors = cp.asarray(neighbors)

In [ ]:
# Radius Nearest Neighbor for (GPU)
%%timeit
dists, idxs, _, _ = frnn.frnn_grid_points(
    points1=cp.asnumpy(queries),
    points2=cp.asnumpy(dataset),
    lengths1=None,
    lengths2=None,
    K=k,
    r=rad,
    grid=None,
    return_nn=False,
    return_sorted=True,
)

3. Plot the times using a barplot.

4. Increase the size of the dataset using the following values: 20,000 50,000 and 100,000. For each value run the 4 knn methods and record the timing. Plot them together with the initial experiments for dataset of size 10,000 and make a summary plot.  

5. Only the SkLearn and frnn can perform a fixed radius search, that doesn't require sorting the result list to make the cut cording with the number of neighbors k. This means that for the fixed radius search sorting is not necessary and it can be performed much faster. Run fixed radius search using these two functions and compare them using a bar plot.

6. Load the dataset sift-128-euclidean from the ann-benchmarks and repeat the same experiments on the dataset using k = 100.

In [ ]:
def load_dataset(dataset_url="http://ann-benchmarks.com/sift-128-euclidean.hdf5", work_folder=None):
    """Download dataset from url. It is expected that the dataset contains a hdf5 file in ann-benchmarks format

    Parameters
    ----------
      dataset_url address of hdf5 file
      work_folder name of the local folder to store the dataset

    """
    dataset_filename = dataset_url.split("/")[-1]

    # We'll need to load store some data in this tutorial
    if work_folder is None:
        work_folder = os.path.join(tempfile.gettempdir(), "raft_example")

    if not os.path.exists(work_folder):
        os.makedirs(work_folder)
    print("The index and data will be saved in", work_folder)

    ## download the dataset
    dataset_path = os.path.join(work_folder, dataset_filename)
    if not os.path.exists(dataset_path):
        urllib.request.urlretrieve(dataset_url, dataset_path)

    f = h5py.File(dataset_path, "r")

    return f

In [ ]:
WORK_FOLDER = os.path.join(tempfile.gettempdir(), "raft_example")
f = load_dataset("http://ann-benchmarks.com/sift-128-euclidean.hdf5", work_folder=WORK_FOLDER)

7 (Extra credit) Implement any approximate search from raft such as CAGRA, HNSW, IVF-FLAT, or IVF-PQ. Are these method providing any speed up over the brut force implementation from raft?